In [3]:
import math
import os
import random

from Bio import SeqIO

In [4]:
# Load data 
fasta_seq = SeqIO.parse("../out/label.fasta", 'fasta')
fasta_label = SeqIO.parse("../out/unmasked_seq_file.fasta", 'fasta')

In [ ]:
# Ratio for splitting into validation, test and train 
train_ratio = .80 
validation_ratio = .10 
test_ratio = .10 

In [ ]:
# List of all protein ids in the data set
for 